# A Custom Fully Convolutional Networks For Time Series Classification

### Importing Three Different Time Series Datasets by using sktime's `load_UCR_UEA_dataset` 
#### About the datasets:
* First dataset is called **DistalPhalanxOutlineCorrect**. It has 2 classes, It is of type Image, and it has one dimension.
* Second dataset is called **ItalyPowerDemand**. It has 2 classes, It is of type Sensor, and it has one dimension.
* Third dataset is called **BME**. It has 3 classes, It is of type Simulated, and it has one dimension.

In [ ]:
# ------ Importing necessary packages!
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# load_UCR_UEA_dataset
from sktime.datasets import load_UCR_UEA_dataset

In [2]:
# ------ Importing Time Series Datasets!
# 2 Classes, 1D, Image
X_train_1, y_train_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="train", return_type="numpy2D")
X_test_1, y_test_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="test", return_type="numpy2D")

# 2 Classes, 1D, Sensor 
X_train_2, y_train_2 = load_UCR_UEA_dataset(name= "ItalyPowerDemand", split="train", return_type="numpy2D")
X_test_2, y_test_2 = load_UCR_UEA_dataset(name="ItalyPowerDemand", split="test", return_type="numpy2D")

# 3Classes, 1D, Simulated
X_train_3, y_train_3 = load_UCR_UEA_dataset(name = "BME", split="train", return_type="numpy2D")
X_test_3, y_test_3 = load_UCR_UEA_dataset(name = "BME", split="test", return_type="numpy2D")

In [3]:
print("Dataset 1:", X_train_1.shape, X_test_1.shape)
print("Dataset 2:", X_train_2.shape, X_test_2.shape)
print("Dataset 3:", X_train_3.shape, X_test_3.shape)

Dataset 1: (600, 80) (276, 80)
Dataset 2: (67, 24) (1029, 24)
Dataset 3: (30, 128) (150, 128)


We have different datasets, which means we must have a function that can take in different datasets with different length.  

The plain baseline is a **Fully Convolutional Networks** which is build by stacking three convolution blocks with the filter sizes {128,256, 128} in each block.  
* Unlike the MCNN and MC-CNN, We exclude any pooling operation.  
* The basic block is a convolutional layer followed by a batch normalization layer and a ReLU activation layer.  
* The convolution operation is fulfilled by three 1-D kernels with the sizes {8, 5, 3} without striding.
* Batch normalization is applied to speed up the convergence speed and help improve generalization. 
* After the convolution blocks, the features are fed into a global average pooling layer instead of a fully connected layer, which largely reduces the number of weights. The final label is produced by a softmax layer


The FCN Model is then trained with **Adam** with learning rate 0.001, ρ = 0.9, β1 = 0.9, β2 = 0.999 and e = 1e − 8!  

The loss function for the model is **categorical cross entropy**! With **accuracy** as metrics!

In [4]:
# We are having different dataset with different types and sizes!
def create_fcn_model(input_shape, num_classes):
    fcn_model = tf.keras.Sequential([
        keras.layers.Conv1D(filters=128, kernel_size=8, input_shape=input_shape),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv1D(filters=256, kernel_size=5),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv1D(filters=128, kernel_size=3),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    fcn_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.001, epsilon=1e-8),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return fcn_model

Defining **callbacks** parameters for fine tuning!  
Defining **label encoder** for converting data to numerical ones.

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
]